# Torch.Autograd를 이용한 자동미분

신경망을 학습할 때 가장 자주 사용되는 알고리즘은 역전파입니다. 이 알고리즘에서, 매개변수(모델 가중치)는 주어진 매개변수에 대한 손실 함수의 변화도(gradient)에 따라 조정됩니다.

이러한 변화도를 계산하기 위해 PyTorch에는 torch.autograd라고 불리는 자동 미분 엔진이 내장되어 있습니다. 이는 모든 계산 그래프에 대한 변화도의 자동 계산을 지원합니다.

입력 x, 매개변수 w와 b , 그리고 일부 손실 함수가 있는 가장 간단한 단일 계층 신경망을 가정하겠습니다. PyTorch에서는 다음과 같이 정의할 수 있습니다:

In [1]:
import torch

x = torch.ones(5) # input tensor
y = torch.zeros(3) # output tensor
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
print(f"{x} x {w} + {b} = {z}")
print(f"{loss} = torch.nn.functional.binary_cross_entropy_with_logits({z}, {y})")

tensor([1., 1., 1., 1., 1.]) x tensor([[ 0.8557,  1.0269,  0.6058],
        [-0.1978, -1.1899, -1.0107],
        [-0.1256,  0.1268, -1.0994],
        [ 1.3848,  0.1957, -0.2494],
        [ 0.7276, -0.1655,  0.5424]], requires_grad=True) + tensor([ 0.6087, -0.1200, -0.0736], requires_grad=True) = tensor([ 3.2533, -0.1260, -1.2849], grad_fn=<AddBackward0>)
1.38919198513031 = torch.nn.functional.binary_cross_entropy_with_logits(tensor([ 3.2533, -0.1260, -1.2849], grad_fn=<AddBackward0>), tensor([0., 0., 0.]))


In [2]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x1077feac0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x1077fec40>


In [3]:
print(w.grad)
print(b.grad)
print(f"{loss.backward()}")
print(w.grad)
print(b.grad)

None
None
None
tensor([[0.3209, 0.1562, 0.0722],
        [0.3209, 0.1562, 0.0722],
        [0.3209, 0.1562, 0.0722],
        [0.3209, 0.1562, 0.0722],
        [0.3209, 0.1562, 0.0722]])
tensor([0.3209, 0.1562, 0.0722])


## 변화도 추적 멈추기
Requires_grad=True인 모든 텐서는 연산 기록을 추적하고 Gradient 계산을 지원함.   
그러나 모델 학습 후 inference 단계에서는 forward propagation만 하면 됨.   
이럴경우 연산 코드를 ```torch.no_grad```블록으로 둘러싸서 연산을 멈출 수 있음.   
또는 ```.detach()```메소드를 사용해서 연산을 멈출 수 있음.

- 순전파 단계
  - 요청된 연산을 수행하여 텐서를 계산
  - DAG(Directed Acyclic Graph)에 연산의 Gradient function을 maintain한다.
- 역전파 단계
  - DAG의 Root에서 .backward가 호출될 때 시작된다.
  - 각 .grad_fn으로 gradient계산.
  - 각 tensor.grad 속성에 계산 결과를 accumulate.
  - 연쇄 법칙을 사용해서 모든 leaf tensor들까지 propagate 한다.   



In [4]:
z - torch.matmul(x, w) + b
print(z.requires_grad)
z_det = z.detach_()
print(z.requires_grad)
print(z_det.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)


True
False
False
False


## Jacobian Product

- tensor.pow(n): 각 인수에 n제곱

In [6]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
print(f"in\b{inp}")
print(f"out\n {out}")
print(f"torch.ones_like(inp): \n{torch.ones_like(inp)}")
out.backward(torch.ones_like(inp), retain_graph=True)
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(inp), retain_graph=True)
print(f"Second call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print(f"Call after zeroing gradients\n{inp.grad}")

itensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]], requires_grad=True)
out
 tensor([[4., 1., 1., 1., 1.],
        [1., 4., 1., 1., 1.],
        [1., 1., 4., 1., 1.],
        [1., 1., 1., 4., 1.],
        [1., 1., 1., 1., 4.]], grad_fn=<PowBackward0>)
torch.ones_like(inp): 
tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])
First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])
Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])
Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
